### Import Libraries

In [1]:
import tda
import pandas as pd
import json
import datetime
import httpx
import math
import time
import random

### Get Date

In [2]:
now = datetime.datetime.now()
yesterday = (now - datetime.timedelta(days = 1)).strftime('%Y%m%d')
today = (now).strftime('%Y%m%d')

current_month = now.month
current_year = now.year

print('Today:\t\t', today)
print('Yesterday:\t', yesterday)
print('Month:\t\t', current_month)
print('Year:\t\t', current_year)

Today:		 20210201
Yesterday:	 20210131
Month:		 2
Year:		 2021


### Create Client

In [3]:
#read file with username, pass, server
with open('tda.json') as f:
    data = json.load(f)

API_KEY = data['api_key']
REDIRECT_URI = data['redirect_uri']
TOKEN_PATH = data['token_path']

def make_webdriver():
    # Import selenium here because it's slow to import
    from selenium import webdriver

    driver = webdriver.Chrome()
    atexit.register(lambda: driver.quit())
    return driver


# Create a new client
client = tda.auth.easy_client(
    API_KEY,
    REDIRECT_URI,
    TOKEN_PATH,
    make_webdriver)

In [4]:
# Load S&P 500 composition from documentation
SP500_URL = "https://tda-api.readthedocs.io/en/latest/_static/sp500.txt"
sp500 = httpx.get(
    SP500_URL, headers={
        "User-Agent": "Mozilla/5.0"}).read().decode().split()

In [5]:
#create empty df
d = {'Symbol':[],
     'Name':[],
     'divDate':[],
     'divAmount':[],
     'divYield':[],
     'closePrice':[],
     'lastPrice':[],
     'expDate1':[],
     'daysUnitlExp1':[],
     'strikePrice1':[],
     'callPremium':[]}
df = pd.DataFrame(data = d)

df

,Symbol,Name,divDate,divAmount,divYield,closePrice,lastPrice,expDate1,daysUnitlExp1,strikePrice1,callPremium


In [6]:
def get_standard_quote(symb):
    symb = symb.upper()
    #get quote
    resp = client.get_quote(symb)
    assert resp.status_code == 200, resp.raise_for_status()
    
    return resp.json()

def get_call_chain(symb):
    resp = client.get_option_chain(symb,
                                   contract_type = tda.client.Client.Options.ContractType.CALL,
                                   strike_range = tda.client.Client.Options.StrikeRange.OUT_OF_THE_MONEY,
                                   strike_count = 5,
                                   strike_from_date = div_date,
                                   strike_to_date = div_date + datetime.timedelta(days = 30)
                                    )
    assert resp.status_code == 200, resp.raise_for_status()
    
    return resp.json()

def myround5(x, base=5):
    return base * math.ceil(x/base)

def myround10(x, base=10):
    return base * math.ceil(x/base)

In [7]:
for x in sp500:
    std_resp_json = get_standard_quote(x)
    if len(std_resp_json) > 0:
        div_amount = std_resp_json[x]['divAmount']/4
        div_yield = std_resp_json[x]['divYield']
        div_date = std_resp_json[x]['divDate']
        close_price = std_resp_json[x]['closePrice']
        last_price = std_resp_json[x]['lastPrice']
        name = std_resp_json[x]['description']
        
        #get option chain if company pays dividend
        if div_amount > 0:
            #convert div_date to datetime
            div_date = datetime.datetime.fromisoformat(std_resp_json[x]['divDate'])
            time.sleep(random.randrange(3))
            
            #only look for option chains with div date in the future
            if div_date > now:
                call_resp_json = get_call_chain(x)

                #list of expirations
                exp_date_list = [a for a in call_resp_json['callExpDateMap'].keys()]
                
                exp_date1a = exp_date_list[0]
                exp_date1b = exp_date_list[0].split(':')[0]
                days_until_exp1 = exp_date_list[0].split(':')[1]
                
                str_price_list = [float(str_pr) for str_pr in call_resp_json['callExpDateMap'][exp_date1a].keys()]
                for i, j in enumerate(str_price_list):
                    if j > last_price:
                        strike_price1 = j
                        break
                
                try:
                    call_premium = call_resp_json['callExpDateMap'][exp_date1a][str(strike_price1)][0]['bid']
                except KeyError:
                    pass
            
            else: 
                exp_date1b = None
                strike_price1 = None
                call_premium = None
                days_until_exp1 = None
        else: 
            exp_date1 = None
            strike_price1 = None
            call_premium = None
        
    symb_div = {'Symbol':x,
                'Name':name,
                'divDate':div_date,
                'divAmount':div_amount,
                'divYield':div_yield,
                'closePrice':close_price,
                'lastPrice':last_price,
                'expDate1':exp_date1b,
                'daysUnitlExp1':days_until_exp1,
                'strikePrice1':strike_price1,
                'callPremium':call_premium
               }

    symb_df = pd.DataFrame.from_dict(symb_div, orient = 'index').T
        
        
    df = pd.concat([df,symb_df])

In [8]:
df.sample(5)

,Symbol,Name,divDate,divAmount,divYield,closePrice,lastPrice,expDate1,daysUnitlExp1,strikePrice1,callPremium
0,AES,The AES Corporation Common Stock,2021-01-28 00:00:00,0.1505,2.47,24.39,25.2232,None,None,None,None
0,SPGI,S&P Global Inc. Common Stock,2021-02-23 00:00:00,0.77,0.97,317,320.35,2021-02-26,25,322.5,7.6
0,STX,Seagate Technology PLC - Ordinary Shares (Irel...,2021-03-23 00:00:00,0.67,4.05,66.12,66.12,2021-04-16,74,67.5,3.45
0,WLTW,Willis Towers Watson Public Limited Company - ...,2020-12-30 00:00:00,0.71,1.4,202.94,207.775,None,None,None,None
0,STZ,"Constellation Brands, Inc. Common Stock",2021-02-08 00:00:00,0.75,1.42,210.93,210.93,2021-02-12,11,212.5,4.4


In [9]:
#convert data types
df['divDate'] = pd.to_datetime(df['divDate'])
df['expDate1'] = pd.to_datetime(df['expDate1'])
df['divAmount'] = df['divAmount'].astype(float)
df['lastPrice'] = df['lastPrice'].astype(float)
df['daysUnitlExp1'] = df['daysUnitlExp1'].astype(float)
df['strikePrice1'] = df['strikePrice1'].astype(float)
df['callPremium'] = df['callPremium'].astype(float)

#calculate cols
df['potentialGainCallExecuted'] = (df['divAmount'] + df['callPremium'] + (df['strikePrice1'] - df['lastPrice'])) * 100
df['cost'] = (df['lastPrice'] - df['callPremium']) * 100

#return call executed
df['returnCallExecuted'] = df['potentialGainCallExecuted'] / df['cost']
df['annualizedReturnCallExecuted'] = (365/df['daysUnitlExp1']) * df['returnCallExecuted']

#return call not executed and stock does not change
df['returnCallNotExecuted'] = (df['divAmount'] + df['callPremium']) * 100 / df['cost']
df['annualizedReturnCallNotExecuted'] = (365/df['daysUnitlExp1']) * df['returnCallNotExecuted']

#break even
df['breakEven'] = (df['cost'] - (df['divAmount']*100))/100

df.reset_index(inplace = True, drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Symbol                           504 non-null    object        
 1   Name                             504 non-null    object        
 2   divDate                          385 non-null    datetime64[ns]
 3   divAmount                        504 non-null    float64       
 4   divYield                         504 non-null    object        
 5   closePrice                       504 non-null    object        
 6   lastPrice                        504 non-null    float64       
 7   expDate1                         117 non-null    datetime64[ns]
 8   daysUnitlExp1                    117 non-null    float64       
 9   strikePrice1                     96 non-null     float64       
 10  callPremium                      96 non-null     float64      

In [10]:
#df_div = df[~df['expDate1'].isna()]
df_div = df[df['divAmount'] > 0]
df_covered_call = df[df['strikePrice1'] > 0]

#export to excel
df_div.to_excel('Output/DividendStocks_' + today + '.xlsx', index = False)
df_covered_call.to_excel('Output/CoveredCalls_' + today + '.xlsx', index = False)
df_div.head()

,Symbol,Name,divDate,divAmount,divYield,closePrice,lastPrice,expDate1,daysUnitlExp1,strikePrice1,callPremium,potentialGainCallExecuted,cost,returnCallExecuted,annualizedReturnCallExecuted,returnCallNotExecuted,annualizedReturnCallNotExecuted,breakEven
0,A,"Agilent Technologies, Inc. Common Stock",2021-01-04,0.194,0.65,120.17,120.17,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAP,Advance Auto Parts Inc Advance Auto Parts Inc W/I,2020-12-17,0.250,0.67,149.14,150.32,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAPL,Apple Inc. - Common Stock,2021-02-05,0.205,0.62,131.96,133.99,2021-02-05,4.0,134.0,2.65,286.5,13134.0,0.021814,1.990492,0.021737,1.983545,131.135
4,ABBV,AbbVie Inc. Common Stock,2021-01-14,1.300,5.07,102.48,103.57,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ABC,AmerisourceBergen Corporation Common Stock,2021-02-11,0.440,1.69,104.2,104.20,2021-02-12,11.0,105.0,2.30,354.0,10190.0,0.034740,1.152734,0.026889,0.892229,101.460
